# Nlp final project

1. Yaniv Gabay - 205745615 - yanivga@edu.hac.ac.il
2. Shahar Asher - 209305408 - shaharas@edu.hac.ac.il
3. Hadar Liel Harush - 211721568 - hadarhar@edu.hac.ac.il
## Eminem song generator

This project will compare different models, all trained on the legendary hip-hop artist Eminem lyrics.
We want to expirment with different models, our own model, different apis, to see and compare the results.


to be added?

In [3]:
######imports

#pip install allennlp
#pip install flair
#pip install pronouncing
import torch
import pandas as pd
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, ELMoEmbeddings,TransformerWordEmbeddings
from transformers import GPT2Model, GPT2Tokenizer
import pronouncing
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Make sure to download NLTK data (e.g., WordNet, Punkt tokenizer models)
nltk.download('wordnet')
nltk.download('punkt')

#### if you want to export data
####
debug = False



c:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yanivg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yanivg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
##### Loading the data
### we be using <startVerse> <endVerse> <startChorus><endChorus>
## to give our model some different between the verses and the choruses.

#important to export the data to see the effect
#of changes, especilly with a small dataset
def export_data(data,stage,debug_mode):
    if(not debug_mode):
        return
    try:
        data.to_csv('data_'+stage+'.csv')
    except Exception as e:
        print("Error exporting the csv file:", e)  
#basic import
def import_data(location):
    df = read_data(location)
    return df
#read with try except,can add more specific
#we try different encoding, you can never know what chars you will find.
# this one is working with latin1, but we still left the function
#exceptions
def read_data(location):
    try_encodings = ['utf-8', 'latin1', 'utf-16', 'cp1252', 'ISO-8859-1', 'windows-1252']
    for encoding in try_encodings:
        try:
            df = pd.read_csv(location ,encoding=encoding,delimiter='\t')
            print(f"Success with encoding: {encoding}")
            print(df.head())
            return df
            break
        except Exception as e:
            print(f"Error with encoding {encoding}: {e}")
####
file_loc = './Eminem_Lyrics.csv'
df = import_data(file_loc)

export_data(df,'afterImport-1',debug)



Error with encoding utf-8: 'utf-8' codec can't decode byte 0x92 in position 6: invalid start byte
Success with encoding: latin1
                        Album_Name       Song_Name  \
0  Music To Be Murdered By: Side B  Alfred (Intro)   
1  Music To Be Murdered By: Side B     Black Magic   
2  Music To Be Murdered By: Side B  Alfreds Theme   
3  Music To Be Murdered By: Side B       Tone Deaf   
4  Music To Be Murdered By: Side B  Book of Rhymes   

                                              Lyrics  \
0  [Intro: Alfred Hitchcock]\nThus far, this albu...   
1  [Chorus: Skylar Grey & Eminem]\nBlack magic, n...   
2  [Verse 1]\nBefore I check the mic (Check, chec...   
3  [Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...   
4  [Intro]\nI don't smile, I don't frown, get too...   

                                           Album_URL   Views  \
0  https://genius.com/albums/Eminem/Music-to-be-m...   24.3K   
1  https://genius.com/albums/Eminem/Music-to-be-m...  180.6K   
2  https://genius.

In [5]:
#actual messing with the dataset
import re


def insert_tokens(lyrics):
    # Pattern to identify all tags (verses, choruses, etc.)
    pattern = re.compile(r'\[(Verse|Chorus|Intro|Outro)')
    
    # Placeholder for processed lyrics
    processed_lyrics = "<startSong>\n"
    last_tag = None

    for line in lyrics.split('\n'):
        tag_match = pattern.match(line)
        if tag_match:
            # Close the previous tag if exists
            if last_tag:
                processed_lyrics += f"<end{last_tag}>\n"
            # Update last_tag and open a new tag
            last_tag = tag_match.group(1)  # Capture the tag name (Verse or Chorus)
            processed_lyrics += f"<start{last_tag}>\n"
        else:
            processed_lyrics += line + "\n"

    # Close the last opened tag
    if last_tag:
        processed_lyrics += f"<end{last_tag}>\n"
    processed_lyrics += "<endSong>"

    return processed_lyrics

def remove_remaining_tags(lyrics):
    # Pattern to match any text within square brackets
    pattern = re.compile(r'\[[^\]]*\]')
    cleaned_lyrics = re.sub(pattern, '', lyrics)
    return cleaned_lyrics

def handle_special(lyrics):
    #remove ' and - and ? and !
    output_string = re.sub(r"['\-?!]",'', lyrics)
    return output_string

new_df = df['Lyrics'].apply(insert_tokens)

print(new_df)
export_data(new_df,'afterInsertTokens',debug)


## some songs can have some another tags, like post-chorus,pre etc. 
## we arent intresnted in those, so we will just delete them in the next step
new_df = new_df.apply(remove_remaining_tags)
new_df = new_df.apply(handle_special)
#lowercase everything
new_df = new_df.str.lower()

export_data(new_df,'afterLowerAndRemove',debug)


0      <startSong>\n<startIntro>\nThus far, this albu...
1      <startSong>\n<startChorus>\nBlack magic, night...
2      <startSong>\n<startVerse>\nBefore I check the ...
3      <startSong>\n<startIntro>\nYeah, I'm sorry (Hu...
4      <startSong>\n<startIntro>\nI don't smile, I do...
                             ...                        
343    <startSong>\n<startChorus>\nI know there's som...
344    <startSong>\n<startIntro>\nYeah, yeah, I get i...
345    <startSong>\n<startVerse>\nI cut back on the s...
346    <startSong>\n<startIntro>\nC'mon!\n\n<endIntro...
347    <startSong>\n<startIntro>\nCDs ain't selling n...
Name: Lyrics, Length: 348, dtype: object


In [6]:
### special chars
def remove_non_ascii_and_print(text):
    cleaned_text = []
    removed_chars = []
    for char in text:
        if ord(char) < 128:
            cleaned_text.append(char)
        else:
            removed_chars.append(char)
    if removed_chars:
        print(f"Removed characters: {''.join(removed_chars)}")
    return ''.join(cleaned_text)
clean = new_df.apply(remove_non_ascii_and_print)


export_data(clean,'afterRemoveNonAscii',debug)

#### we still have (lyrics) in the dataset, need to decide
### we still have ! or ? in the dataset, should we remove? 

Removed characters: 
Removed characters: á
Removed characters: 
Removed characters: 
Removed characters: ó
Removed characters: 
Removed characters: 
Removed characters: éé
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: ö
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: éâé
Removed characters: ó
Removed characters: 
Removed characters: áó
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: óéáí
Removed characters: 
Removed characters: ú
Removed characters: 
Removed characters: â
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters:

## we will try several tokenizer options

In [7]:
temp_switch = {'<startverse>':'startverse',
                '<endverse>':'endverse',
                '<startchorus>':'startchorus',
                '<endchorus>':'endchorus',
                '<startoutro>':'startoutro',
                '<endoutro>':'endoutro',
                '<startintro>':'startintro',
                '<endintro>':'endintro',
                '<startsong>':'startsong',
                '<endsong>':'endsong'}
def replace_tags(text, temp_switch, reverse=False):
    if reverse:
        temp_switch = {v: k for k, v in temp_switch.items()}
    
    for tag, replacement in temp_switch.items():
        text = text.replace(tag, replacement)
    return text

temp_clean = clean.apply(lambda x: replace_tags(x, temp_switch, reverse=False))
export_data(temp_clean,'temp_clean',debug)
#pip install spacy
import spacy
#python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
spacy_tokens_df = temp_clean.apply(lambda x: [token.text for token in nlp(x)])


import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
nltk_tokens_df = temp_clean.apply(word_tokenize)


from transformers import BertTokenizer
# For BERT
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
num_of_added_tokens = tokenizer_bert.add_special_tokens({'additional_special_tokens':
                                    ['startverse', 'endverse',
                                      'startchorus', 'endchorus',
                                        'startoutro', 'endoutro', 
                                        'startintro', 'endintro',
                                        'startsong', 'endsong']})
print(f"Number of added tokens: {num_of_added_tokens}")
BERT_tokens_df = temp_clean.apply(lambda x: tokenizer_bert.tokenize(x))
from transformers import GPT2Tokenizer
# For GPT-2

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
num_of_added_tokens = tokenizer_gpt2.add_special_tokens({'additional_special_tokens':
                                    ['startverse', 'endverse',
                                      'startchorus', 'endchorus',
                                        'startoutro', 'endoutro', 
                                        'startintro', 'endintro',
                                          'startsong', 'endsong']})
gpt2_tokens_df = temp_clean.apply(lambda x: tokenizer_gpt2.tokenize(x))

export_data(spacy_tokens_df,'spacy',debug)
export_data(nltk_tokens_df,'nltk',debug)
export_data(BERT_tokens_df,'BERT',debug)
export_data(gpt2_tokens_df,'GPT2', debug )

total_token_models = [spacy_tokens_df, nltk_tokens_df, BERT_tokens_df, gpt2_tokens_df]
for model in total_token_models:
    print("")
    print(model.head())

## pronouncing library
## we will use it to get the phonetic features of the words
    

import pronouncing

def get_phonetic_features(word):
    phones = pronouncing.phones_for_word(word)
    stress = [pronouncing.stresses(p) for p in phones] if phones else ''
    return {'phones': phones, 'stress': stress}







spacy_phonetics_df = spacy_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
nltk_phonetics_df = nltk_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
bert_phonetics_df = BERT_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
gpt2_phonetics_df = gpt2_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])

export_data(spacy_phonetics_df, 'spacy_phonetics',debug)
export_data(nltk_phonetics_df, 'nltk_phonetics',  debug)
export_data(bert_phonetics_df, 'bert_phonetics', debug)
export_data(gpt2_phonetics_df, 'gpt2_phonetics', debug)

total_phonetic_models = [spacy_phonetics_df, nltk_phonetics_df, bert_phonetics_df, gpt2_phonetics_df]


##### now , we need to perpare the data for the embeddings
### so padding, indexing unique tokens, etc.

Number of added tokens: 10

0    [startsong, \n, startintro, \n, thus, far, ,, ...
1    [startsong, \n, startchorus, \n, black, magic,...
2    [startsong, \n, startverse, \n, before, i, che...
3    [startsong, \n, startintro, \n, yeah, ,, i, m,...
4    [startsong, \n, startintro, \n, i, do, nt, smi...
Name: Lyrics, dtype: object

0    [startsong, startintro, thus, far, ,, this, al...
1    [startsong, startchorus, black, magic, ,, nigh...
2    [startsong, startverse, before, i, check, the,...
3    [startsong, startintro, yeah, ,, im, sorry, (,...
4    [startsong, startintro, i, dont, smile, ,, i, ...
Name: Lyrics, dtype: object

0    [startsong, startintro, thus, far, ,, this, al...
1    [startsong, startchorus, black, magic, ,, nigh...
2    [startsong, startverse, before, i, check, the,...
3    [startsong, startintro, yeah, ,, im, sorry, (,...
4    [startsong, startintro, i, don, ##t, smile, ,,...
Name: Lyrics, dtype: object

0    [startsong, startintro, thus, Ġfar, ,, Ġthis, ...
1    

In [9]:
from flair.embeddings import FlairEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.data import Sentence
from tqdm import tqdm  # tqdm is a library to display progress bars
import pandas as pd

# Clear GPU memory
#torch.cuda.empty_cache()

# Define stacked embeddings
stacked_embeddings = StackedEmbeddings([
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
])

# Initialize a list to store embeddings
embeddings_cache = []

# Process each song's tokens sequentially
for tokens in tqdm(spacy_tokens_df, desc="Processing songs"):
    sentence = Sentence(' '.join(tokens))
    stacked_embeddings.embed(sentence)
    embeddings_cache.append([token.embedding for token in sentence])

# Convert the list of embeddings to a pandas Series (or DataFrame) aligned with your original DataFrame's index
spacy_embed = pd.Series(embeddings_cache, index=spacy_tokens_df.index)

print(spacy_embed.head(5))

Processing songs:   4%|▍         | 15/348 [06:19<2:20:34, 25.33s/it]


KeyboardInterrupt: 